# Making Model
- This Model is build using python 3.8.10
- different version of python can be impact to the erroneous of the code

In [15]:
# Warning python version < 3.12.0( 3.8.10 recommended)

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from setuptools import setup # use instead of distutils(deprecated)
import tensorflow as tf
# from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]


# Load data

In [17]:
train = pd.read_csv('./archive/written_name_train_v2.csv')
valid = pd.read_csv('./archive/written_name_validation_v2.csv')
test = pd.read_csv('./archive/written_name_test_v2.csv')

train_len = len(train)
test_len = len(test)
validation_len = len(valid)

sum = train_len + test_len + validation_len

print("train data:  {:.2f}  {}".format((train_len/ sum), train_len))
print("valid data:  {:.2f}  {}".format((validation_len / sum), validation_len))
print("test data:   {:.2f}  {}".format((test_len / sum), test_len))

train data:  0.80  330961
valid data:  0.10  41370
test data:   0.10  41370


#  View data

In [ ]:
plt.figure(figsize=(10, 10))

for i in range(4):
    ax = plt.subplot(2, 2, i+1)
    img_dir = './test_v2/test/' + test.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    plt.imshow(image, cmap='gray')
    plt.title(test.loc[i, 'IDENTITY'], fontsize=12)
    plt.axis('off')

plt.subplots_adjust(wspace=0.4, hspace=-0.8)
plt.show()

# Clean data

In [ ]:
print("Number of NaNs in train set      : ", train.IDENTITY.isna().sum())
print("Number of NaNs in validation set : ", valid.IDENTITY.isna().sum())
print("Number of NaNs in validation set : ", test.IDENTITY.isna().sum())

In [ ]:
# Drop the NaNs labels
train.dropna(axis=0, inplace=True)
valid.dropna(axis=0, inplace=True)
test.dropna(axis=0, inplace=True)

In [ ]:
# Remove the Unreadable labels
train = train[train.IDENTITY != 'UNREADABLE']
valid = valid[valid.IDENTITY != 'UNREADABLE']
test = test[test.IDENTITY != 'UNREADABLE']

In [ ]:
# Covert lowercase to uppercase
train.IDENTITY = train.IDENTITY.str.upper()
valid.IDENTITY = valid.IDENTITY.str.upper()
test.IDENTITY = test.IDENTITY.str.upper()

In [ ]:
# Reset the index
train.reset_index(inplace = True, drop=True)
valid.reset_index(inplace = True, drop=True)
test.reset_index(inplace = True, drop=True)

In [ ]:
train_size = len(train)# train 5000 images 
#test_size = len(test)
valid_size = 5000#len(valid)# train 620 images

In [ ]:
valid_batch_size = 620
train_batch_size = 5000
print(train_size, valid_size)

# Preprocess the images

In [ ]:
# Fuction to preprocess the img
def preprocess(img):
    (h, w) = img.shape                                    # Getting the height & width of the image
    
    final_img = np.ones([64, 256])*255                    # Blank white image
    
    # crop    
    if h > 64:
        img = img[:64, :]                                 # If the h>64 then it is cropped to 64
        
    if w > 256:
        img = img[:, :256]                                # If the w>256 then it is cropped to 256
    
    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE) # Rotate 90° Clockwise & return

Preprocess & save test data

In [ ]:
train_x = []

for i in range(train_size):
    img_dir = './archive//train_v2/train/' + train.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    train_x.append(image)
    
    if (i + 1) % 10000 == 0:
        print(f"Processed {i + 1} images")

print("Processing complete.")

In [ ]:
# Split the train_x array into batches of train_batch_size images each
batch_size = train_batch_size
num_batches = len(train_x) // batch_size
remainder = len(train_x) % batch_size

start_idx = 0
for i in range(num_batches):
    end_idx = start_idx + batch_size
    # Extract a batch of images
    batch_images = train_x[start_idx:end_idx]
    # Reshape the batch into the required format (num_images, height, width, channels)
    batch_images = np.array(batch_images).reshape(-1, 256, 64, 1)

    # Create a folder for the current batch
    folder_path = os.path.join('dataset', f'{i + 1}')

    # Save the batch to a .npy file
    filename = os.path.join(folder_path, f'train_x.npy')
    np.save(filename, batch_images)

    start_idx = end_idx
    
    print(i+1)

# Process the remaining images
if remainder > 0:
    # Extract the remaining images
    batch_images = train_x[start_idx:]
    # Reshape the remaining images into the required format
    batch_images = np.array(batch_images).reshape(-1, 256, 64, 1)
    # Create a folder for the remaining images
    folder_path = os.path.join('dataset', f'{num_batches + 1}')
    os.makedirs(folder_path, exist_ok=True)
    # Save the remaining images to a .npy file
    filename = os.path.join(folder_path, f'train_x.npy')
    np.save(filename, batch_images)

In [ ]:
# Convert 'train_x' to a NumPy array and reshape it
#train_x = np.array(train_x).reshape(-1, 256, 64, 1)

#np.save('preprocessed_train_v2', train_x)
train_x = np.load('preprocessed_train_v2.npy')

Preprocess & save valid data

In [ ]:
valid_x = []

for i in range(valid_size):
    img_dir = '.archive/validation_v2/validation/' + valid.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    valid_x.append(image)
    
    if (i + 1) % 1000 == 0:
        print(f"Processed {i + 1} images")

print("Processing complete.")

In [ ]:
# Split the valid_x array into batches of valid_batch_size images each
batch_size = valid_batch_size
num_batches = len(valid_x) // batch_size
remainder = len(valid_x) % batch_size

start_idx = 0
for i in range(num_batches):
    end_idx = start_idx + batch_size
    # Extract a batch of images
    batch_images = valid_x[start_idx:end_idx]
    # Reshape the batch into the required format (num_images, height, width, channels)
    batch_images = np.array(batch_images).reshape(-1, 256, 64, 1)

    # Create a folder for the current batch
    folder_path = os.path.join('dataset', f'{i + 1}')
    os.makedirs(folder_path, exist_ok=True)

    # Save the batch to a .npy file
    filename = os.path.join(folder_path, f'valid_x.npy')
    np.save(filename, batch_images)

    start_idx = end_idx

    print(i+1)
# Process the remaining images
if remainder > 0:
    # Extract the remaining images
    batch_images = valid_x[start_idx:]
    # Reshape the remaining images into the required format
    batch_images = np.array(batch_images).reshape(-1, 256, 64, 1)
    # Create a folder for the remaining images
    folder_path = os.path.join('dataset', f'{num_batches + 1}')
    os.makedirs(folder_path, exist_ok=True)
    # Save the remaining images to a .npy file
    filename = os.path.join(folder_path, f'valid_x.npy')
    np.save(filename, batch_images)

In [ ]:
# Convert 'valid_x' to a NumPy array and reshape it
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)
# np.save('preprocessed_validation_v2', valid_x)
#valid_x = np.load('preprocessed_validation_v2.npy')

# Data Label Preprocess
In this section of the code, we perform data label preprocessing for a text recognition task. The goal is to convert text labels into a numerical format for use in train a neural network. Let's break down the key components of this preprocess:

Character Set and Constants
* alphabets: This string represents the set of valid characters that can appear in the text labels. It includes uppercase letters, hyphen, and space.

* max_str_len: This constant defines the maximum length of the input labels.

* num_of_characters: The number of unique characters, including an extra one for the CTC pseudo-blank character.

* num_of_timestamps: The maximum length of predicted labels.

# Label-to-Number Conversion
* label_to_num(label): This function converts a text label to a numerical representation. It initializes an empty list to store character indices and iterates over each character in the input label. For each character, it finds the index in the alphabets string and appends it to the list. The function returns a NumPy array with the numerical representation of the label.
* Number-to-Label Conversion
num_to_label(num): This function converts a list of numerical values back to a text label. It iterates over each character index in the input list, checks for the CTC blank character (-1), and appends the corresponding character from alphabets to the output string.

# Initialization of Arrays
Several arrays are initialized to store the preprocessed labels and related information for both training and validation datasets. These include:

* train_y: An array to store the converted labels for the training dataset.

* train_label_len: An array to store the length of labels for each data point in the training dataset.

* train_input_len: An array to store the input length for each data point in the training dataset.

* train_output: An array to store training data outputs (usually used for CTC loss).

# Data Processing
A loop is used to process both the training and validation datasets. Within the loop:

* The 'IDENTITY' value from the dataset is retrieved and converted to a string if it's not already.

* The label length is set and label values are stored in the corresponding arrays (train_y for training and valid_y for validation).

In [ ]:
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "  # The set of valid characters
max_str_len = 34                             # Maximum length of input labels
num_of_characters = len(alphabets) + 1       # Number of unique characters, plus 1 for CTC pseudo-blank
num_of_timestamps = 64                       # The maximum length of predicted labels

In [ ]:
def label_to_num(label):
    return np.array([alphabets.find(ch) if ch in alphabets else 27 for ch in label])

In [ ]:
def num_to_label(num):
    return ''.join([alphabets[ch] for ch in num if ch != -1])

In [ ]:
# Example
name = 'TATIA'
name_num = label_to_num(name)
print(name_num,'\n', num_to_label(name_num))

In [ ]:
blank_label = -1

In [ ]:
# Initialize arrays
train_y = np.ones([train_size, max_str_len]) * blank_label
train_label_len = np.zeros([train_size, 1])
train_input_len = np.ones([train_size, 1]) * (num_of_timestamps - 2)
train_output = np.zeros([train_size])

# Loop through the training data
for i in range(train_size):
    # Get the 'IDENTITY' value and convert it to a string if it's not already
    identity = train.loc[i, 'IDENTITY']
    if not isinstance(identity, str):
        identity = str(identity)
    
    # Set the label length and label values in train_y
    train_label_len[i] = len(identity)
    
    # Assuming label_to_num is a function that converts characters to numerical values
    train_y[i, 0:len(identity)] = label_to_num(identity)

In [ ]:
batch_size = train_batch_size

# Iterate through each folder
for i in range(1, 67):
    folder_path = os.path.join('dataset', str(i))

    batch_train_y = np.ones([batch_size, max_str_len]) * blank_label
    batch_train_label_len = np.zeros([batch_size, 1])
    batch_train_input_len = np.ones([batch_size, 1]) * (num_of_timestamps - 2)
    batch_train_output = np.zeros([batch_size])
    
    # Loop through the training data
    for j in range(batch_size):
        # Get the 'IDENTITY' value and convert it to a string if it's not already
        identity = train.loc[(batch_size * (i-1)) + j, 'IDENTITY']
        if not isinstance(identity, str):
            identity = str(identity)
        
        # Set the label length and label values in train_y
        batch_train_label_len[j] = len(identity)
        
        # Assuming label_to_num is a function that converts characters to numerical values
        batch_train_y[j, 0:len(identity)] = label_to_num(identity)

    # Save data in the folder
    np.save(os.path.join(folder_path, 'train_y.npy'), batch_train_y)
    np.save(os.path.join(folder_path, 'train_label_len.npy'), batch_train_label_len)
    np.save(os.path.join(folder_path, 'train_input_len.npy'), batch_train_input_len)
    np.save(os.path.join(folder_path, 'train_output.npy'), batch_train_output)
    
    print(i)

In [ ]:
# Initialize arrays for validation data
valid_y = np.ones([valid_size, max_str_len]) * blank_label
valid_label_len = np.zeros([valid_size, 1])
valid_input_len = np.ones([valid_size, 1]) * (num_of_timestamps - 2)
valid_output = np.zeros([valid_size])

# Loop through the validation data
for i in range(valid_size):
    # Get the 'IDENTITY' value and convert it to a string if it's not already
    identity = valid.loc[i, 'IDENTITY']
    if not isinstance(identity, str):
        identity = str(identity)
    
    # Set the label length and label values in valid_y
    valid_label_len[i] = len(identity)
    
    # Assuming label_to_num is a function that converts characters to numerical values
    valid_y[i, 0:len(identity)] = label_to_num(identity)

In [ ]:
batch_size = valid_batch_size

# Iterate through each folder
for i in range(1, 67):
    folder_path = os.path.join('dataset', str(i))

    batch_valid_y = np.ones([batch_size, max_str_len]) * blank_label
    batch_valid_label_len = np.zeros([batch_size, 1])
    batch_valid_input_len = np.ones([batch_size, 1]) * (num_of_timestamps - 2)
    batch_valid_output = np.zeros([batch_size])
    
    # Loop through the training data
    for j in range(batch_size):
        # Get the 'IDENTITY' value and convert it to a string if it's not already
        identity = valid.loc[((batch_size * (i-1)) + j), 'IDENTITY']
        if not isinstance(identity, str):
            identity = str(identity)
        # Set the label length and label values in train_y
        batch_valid_label_len[j] = len(identity)
        
        # Assuming label_to_num is a function that converts characters to numerical values
        batch_valid_y[j, 0:len(identity)] = label_to_num(identity)

    # Save data in the folder
    np.save(os.path.join(folder_path, 'valid_y.npy'), batch_valid_y)
    np.save(os.path.join(folder_path, 'valid_label_len.npy'), batch_valid_label_len)
    np.save(os.path.join(folder_path, 'valid_input_len.npy'), batch_valid_input_len)
    np.save(os.path.join(folder_path, 'valid_output.npy'), batch_valid_output)
    
    print(i)

In [ ]:
print('True label : ',train.loc[4, 'IDENTITY'] , '\ntrain_y : ',train_y[4],'\ntrain_label_len : ',train_label_len[4], 
      '\ntrain_input_len : ', train_input_len[4])

In [ ]:
# Define the input layer with a shape of (256, 64, 1) for grayscale images
input_data = Input(shape=(256, 64, 1), name='input')

# Convolutional Layer 1: 32 filters, (3, 3) kernel, 'same' padding, He normal initialization
inner = Conv2D(32, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)
inner = BatchNormalization()(inner)                         # Batch normalization
inner = Activation('relu')(inner)                           # ReLU activation
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)  # Max-pooling

# Convolutional Layer 2: 64 filters, (3, 3) kernel, 'same' padding, He normal initialization
inner = Conv2D(64, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)  
inner = Activation('relu')(inner)     
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)  
inner = Dropout(0.3)(inner)

# Convolutional Layer 3: 128 filters, (3, 3) kernel, 'same' padding, He normal initialization
inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)                         
inner = Activation('relu')(inner)                           
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)  
inner = Dropout(0.3)(inner)                                 

# Reshape the output for sequence processing
inner = Reshape(target_shape=((64, 1024)), name='reshape')(inner)

# Fully Connected Layer 1: 64 units, ReLU activation, He normal initialization
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)

# Bidirectional LSTM Layers: 256 units, return sequences
inner = Bidirectional(LSTM(256, return_sequences=True), name='lstm1')(inner)
inner = Bidirectional(LSTM(256, return_sequences=True), name='lstm2')(inner)

# Output Layer: Number of characters, He normal initialization
inner = Dense(num_of_characters, kernel_initializer='he_normal', name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)  # Softmax activation

In [ ]:
# Create the model with input and output layers
model = Model(inputs=input_data, outputs=y_pred)

model.load_weights('checkpoints/model_14_1.221.h5')

In [ ]:
# Display a summary of the model architecture
model.summary()

Compute the CTC loss between predicted and true labels.

In [ ]:
# The ctc loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # The 2 is critical here since the first couple outputs of the RNN tend to be garbage
    y_pred = y_pred[:, 2:, :]
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
# Define input placeholders for true labels, input sequence length, and label sequence length
labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

# Calculate CTC loss using the ctc_lambda_func function
ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

# Create the final model that takes input data, true labels, input length, and label length
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [ ]:
# Compile the final model with a dummy loss lambda function (loss calculation occurs elsewhere)
# The optimizer used is Adam with a learning rate of 0.0001
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(learning_rate=0.0001))
history = []

# Train the Model

In [ ]:
# Define the folder to save the model
checkpoint_dir = 'checkpoints/'
# Create the folder if it doesn't exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Define EarlyStopping and ModelCheckpoint callbacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)  # Stop training if val_loss does not improve
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_{epoch:02d}_{val_loss:.3f}.h5'),  # Path to the model file to be saved
    monitor='val_loss',  # Monitor val_loss
    save_best_only=True,  # Save only the best model
    mode='min'  # Save the model when val_loss is at its minimum
)

# Iterate through each folder
for i in range(0, 67):
    folder_path = os.path.join('dataset', str(i))
    
    # Load data from files in the folder
    train_x = np.load(os.path.join(folder_path, 'train_x.npy'))
    train_y = np.load(os.path.join(folder_path, 'train_y.npy'))
    train_input_len = np.load(os.path.join(folder_path, 'train_input_len.npy'))
    train_label_len = np.load(os.path.join(folder_path, 'train_label_len.npy'))
    train_output = np.load(os.path.join(folder_path, 'train_output.npy'))

    valid_x = np.load(os.path.join(folder_path, 'valid_x.npy'))
    valid_y = np.load(os.path.join(folder_path, 'valid_y.npy'))
    valid_input_len = np.load(os.path.join(folder_path, 'valid_input_len.npy'))
    valid_label_len = np.load(os.path.join(folder_path, 'valid_label_len.npy'))
    valid_output = np.load(os.path.join(folder_path, 'valid_output.npy'))

    # Train the model with the current data
    print("dataset ", i)
    history_buff = model_final.fit(
        x=[train_x, train_y, train_input_len, train_label_len],
        y=train_output,
        validation_data=([valid_x, valid_y, valid_input_len, valid_label_len], valid_output),
        epochs=20,
        batch_size=64,
        callbacks=[early_stopping_callback, checkpoint_callback]
    )
    
    history.extend(history_buff.history['loss'])
    history.extend(history_buff.history['val_loss']) 
    
     # Delete variables to free up memory
    del train_x, train_y, train_input_len, train_label_len, train_output
    del valid_x, valid_y, valid_input_len, valid_label_len, valid_output


In [ ]:
class CustomDataLoader(tf.keras.utils.Sequence):
    def __init__(self, batch_size):
        self.batch_size = batch_size
    
        self.train_x = train_x
        self.train_y = train_y
        self.train_input_len = train_input_len
        self.train_label_len = train_label_len
        self.train_output = train_output

    def __len__(self):
        return len(self.train_x) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.train_x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.train_y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_input_len = self.train_input_len[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_label_len = self.train_label_len[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_output = self.train_output[idx * self.batch_size:(idx + 1) * self.batch_size]
    
        return ([batch_x, batch_y, batch_input_len, batch_label_len], batch_output)
    
    def on_epoch_end(self):
        
        pass
    
batch_size = 32

data_loader = CustomDataLoader(batch_size)


In [ ]:
class ValidationDataLoader(tf.keras.utils.Sequence):
    def __init__(self, valid_x, valid_y, valid_input_len, valid_label_len, valid_output, batch_size=32):
        self.valid_x = valid_x
        self.valid_y = valid_y
        self.valid_input_len = valid_input_len
        self.valid_label_len = valid_label_len
        self.valid_output = valid_output
        self.batch_size = batch_size

    def __len__(self):
        return len(self.valid_x) // self.batch_size

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size
        batch_x = self.valid_x[start_idx:end_idx]
        batch_y = self.valid_y[start_idx:end_idx]
        batch_input_len = self.valid_input_len[start_idx:end_idx]
        batch_label_len = self.valid_label_len[start_idx:end_idx]
        batch_output = self.valid_output[start_idx:end_idx]
        return ([batch_x, batch_y, batch_input_len, batch_label_len], batch_output)


batch_size = 32

valid_loader = ValidationDataLoader(valid_x, valid_y, valid_input_len, valid_label_len, valid_output, batch_size)

In [ ]:
import pickle

# save DataLoader
with open('data_loader.pkl', 'wb') as f:
    pickle.dump(data_loader, f)


In [ ]:
# load DataLoader
with open('data_loader.pkl', 'rb') as f:
    loaded_data_loader = pickle.load(f)

In [ ]:
validation_data = data_loader.get_validation_data(batch_size=32)
x_data, y_data = validation_data[0], validation_data[1]

for data_array in x_data:
    print("Shape of data array:", data_array.shape)

print("Shape of output array:", y_data.shape)


In [ ]:
checkpoint_dir = 'checkpoints/'
# Define EarlyStopping and ModelCheckpoint callbacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)  # Stop training if val_loss does not improve
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_{epoch:02d}_{val_loss:.3f}.h5'),  # Path to the model file to be saved
    monitor='val_loss',  # Monitor val_loss
    save_best_only=True,  # Save only the best model
    mode='min'  # Save the model when val_loss is at its minimum
)


history = model_final.fit(data_loader,
                          validation_data= valid_loader,
                          epochs=80,
                          callbacks=[early_stopping_callback, checkpoint_callback])


In [ ]:
#  loss and val_loss history
loss = history[::2]
val_loss = history[1::2]

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Save model 
model.save('trained_model_14_3.h5')

In [ ]:
# save history
with open('history12.pkl', 'wb') as file:
    pickle.dump(history, file)

In [ ]:
#Load model
model = load_model('trained_model.h12')

In [ ]:
preds = model.predict(valid_x)
decoded = tf.keras.backend.get_value(tf.keras.backend.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

prediction = []
for i in range(valid_size):
    prediction.append(num_to_label(decoded[i]))

In [ ]:
y_true = valid.loc[0:valid_size, 'IDENTITY']
correct_char = 0
total_char = 0
correct = 0

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 

print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

# Prediction on Test Set

In [ ]:
test = pd.read_csv('written_name_test_v2.csv')

plt.figure(figsize=(15, 10))
for i in range(6):
    ax = plt.subplot(2, 3, i+1)
    img_dir = './test_v2/test/'+ test.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    plt.imshow(image, cmap='gray')
    
    image = preprocess(image)
    image = image/255.
    pred = model.predict(image.reshape(1, 256, 64, 1))
    decoded = tf.keras.backend.get_value(tf.keras.backend.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], 
                                       greedy=True)[0][0])
    plt.title(num_to_label(decoded[0]), fontsize=12)
    plt.axis('off')
    
plt.subplots_adjust(wspace=0.2, hspace=-0.8)

In [ ]:
# Initialize variables to track correct characters
correct_characters = 0
total_characters = 0
max_images = 300  # Specify the number of images to evaluate

# Iterate through the test set
for i in range(min(len(test), max_images)):
    # Get the ground truth label
    ground_truth = test.loc[i, 'IDENTITY']
    
    # Preprocess and predict the image using your model
    img_dir = './test_v2/test/' + test.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0
    pred = model.predict(image.reshape(1, 256, 64, 1))
    decoded = tf.keras.backend.get_value(tf.keras.backend.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], greedy=True)[0][0])
    predicted_label = num_to_label(decoded[0])
    
     # Calculate character-level accuracy
    for j in range(min(len(ground_truth), len(predicted_label))):
        if ground_truth[j] == predicted_label[j]:
            correct_characters += 1
        total_characters += 1
        
    print(i)

# Calculate character-level accuracy
character_level_accuracy = (correct_characters / total_characters) * 100

In [ ]:
print('Character-Level Accuracy for {} images: {:.2f}%'.format(max_images, character_level_accuracy))

In [ ]:
# Initialize a dictionary to track errors for each character
character_errors = {}
max_images = 500
# Iterate through the test set
for i in range(min(len(test), max_images)):
    # Get the ground truth label
    ground_truth = test.loc[i, 'IDENTITY']
    
    # Preprocess and predict the image using your model
    img_dir = './test_v2/test/' + test.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0
    pred = model.predict(image.reshape(1, 256, 64, 1))
    decoded = tf.keras.backend.get_value(tf.keras.backend.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], greedy=True)[0][0])
    predicted_label = num_to_label(decoded[0])
    
    # Calculate errors for each character
    for j in range(min(len(ground_truth), len(predicted_label))):
        if ground_truth[j] != predicted_label[j]:
            if ground_truth[j] not in character_errors:
                character_errors[ground_truth[j]] = 1
            else:
                character_errors[ground_truth[j]] += 1
                
# Print characters with the most errors
sorted_errors = sorted(character_errors.items(), key=lambda x: x[1], reverse=True)
print("Character\tErrors")
for char, errors in sorted_errors:
    print(f"{char}\t{errors}")


In [ ]:
print(sorted_errors)

In [ ]:

first_12_chars = sorted_errors[0][0][:17]

train_indexes = []
valid_indexes = []


for idx, row in train.iterrows():
    count = row['IDENTITY'][:12].count(first_12_chars)
    if count > 2:
        train_indexes.append(idx)

for idx, row in valid.iterrows():
    count = row['IDENTITY'][:12].count(first_12_chars)
    if count > 2:
        valid_indexes.append(idx)

print("Train indexes with more than 2 occurrences of the first 12 characters:", train_indexes)
print("Validation indexes with more than 2 occurrences of the first 12 characters:", valid_indexes)


In [ ]:
train_indexes_size = len(train_indexes)
print(train_indexes_size)

In [ ]:
valid_indexes_size = len(valid_indexes)
print(valid_indexes_size)

In [ ]:
train_x = []

for i in train_indexes:
    img_dir = './train_v2/train/' + train.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    train_x.append(image)
    
    
print(f"Processed {len(train_indexes)} images")

print("Processing complete.")

In [ ]:
# create new data set 66

# Reshape the batch into the required format (num_images, height, width, channels)
train_x = np.array(train_x).reshape(-1, 256, 64, 1)

# Create a folder for the current batch
folder_path = os.path.join('dataset', f'{66}')

# Save the batch to a .npy file
filename = os.path.join(folder_path, f'train_x.npy')
np.save(filename, train_x)

In [ ]:
valid_x = []

for i in valid_indexes:
    img_dir = '.archive/validation_v2/validation/' + valid.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    valid_x.append(image)
    
print(f"Processed {len(valid_x)} images")

In [ ]:
# create new data set 66

# Reshape the batch into the required format (num_images, height, width, channels)
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)

# Create a folder for the current batch
folder_path = os.path.join('dataset', f'{66}')

# Save the batch to a .npy file
filename = os.path.join(folder_path, f'valid_x.npy')
np.save(filename, valid_x)

In [ ]:
# Initialize arrays
train_y = np.ones([train_indexes_size, max_str_len]) * blank_label
train_label_len = np.zeros([train_indexes_size, 1])
train_input_len = np.ones([train_indexes_size, 1]) * (num_of_timestamps - 2)
train_output = np.zeros([train_indexes_size])

# Loop through the training data
i = 0
for idx in train_indexes:
    # Get the 'IDENTITY' value and convert it to a string if it's not already
    identity = train.loc[idx, 'IDENTITY']
    if not isinstance(identity, str):
        identity = str(identity)
    
    # Set the label length and label values in train_y
    train_label_len[i] = len(identity)
    
    # Assuming label_to_num is a function that converts characters to numerical values
    train_y[i, 0:len(identity)] = label_to_num(identity)
    i = i + 1

In [ ]:
# Save data in the folder
folder_path = 'dataset/66'

np.save(os.path.join(folder_path, 'train_y.npy'), train_y)
np.save(os.path.join(folder_path, 'train_label_len.npy'), train_label_len)
np.save(os.path.join(folder_path, 'train_input_len.npy'), train_input_len)
np.save(os.path.join(folder_path, 'train_output.npy'), train_output)

In [ ]:
# Initialize arrays
valid_y = np.ones([valid_indexes_size, max_str_len]) * blank_label
valid_label_len = np.zeros([valid_indexes_size, 1])
valid_input_len = np.ones([valid_indexes_size, 1]) * (num_of_timestamps - 2)
valid_output = np.zeros([valid_indexes_size])

# Loop through the training data
i = 0
for idx in valid_indexes:
    # Get the 'IDENTITY' value and convert it to a string if it's not already
    identity = valid.loc[idx, 'IDENTITY']
    if not isinstance(identity, str):
        identity = str(identity)
    
    # Set the label length and label values in train_y
    valid_label_len[i] = len(identity)
    
    # Assuming label_to_num is a function that converts characters to numerical values
    valid_y[i, 0:len(identity)] = label_to_num(identity)
    i = i + 1

In [ ]:
# Save data in the folder
folder_path = 'dataset/66'

np.save(os.path.join(folder_path, 'valid_y.npy'), valid_y)
np.save(os.path.join(folder_path, 'valid_label_len.npy'), valid_label_len)
np.save(os.path.join(folder_path, 'valid_input_len.npy'), valid_input_len)
np.save(os.path.join(folder_path, 'valid_output.npy'), valid_output)